About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [107]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [108]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [109]:
data = pd.read_csv('/content/train.csv (2).zip',index_col=0)
data.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [110]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [111]:
# loading the dataset to a pandas DataFrame
data= pd.read_csv('/content/train.csv (2).zip')

In [60]:
data.shape

(20800, 5)

In [112]:
# print the first 5 rows of the datafr
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [113]:
# counting the number of missing values in the dataset
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [114]:
# replacing the null values with empty st
data= data.fillna('')

In [115]:
# Shuffling
data = data.sample(frac=1)
data.reset_index(inplace=True)
data.drop(["index"], axis=1, inplace=True)

In [116]:
# merging the author name and news title
data['content'] = data['author']+' '+data['title']

In [117]:
print(data['content'])

0        Joseph Goldstein Alt-Right Gathering Exults in...
1        John Koblin Billy Bush, Host on ‘Today,’ Is Su...
2                    Mark Bowytz Let's Eat - The Daily WTF
3        Natalie Dickinson Stephen King Just Got Real W...
4        Charlie Spiering DNC Chair Tom Perez Joins Pro...
                               ...                        
20795    Pam Key Dem Rep Schiff: Nothing Definitive on ...
20796    James Poniewozik Review: ‘The Night Of’ Takes ...
20797    Daniel Nussbaum Smoke a Cigar with Dennis Prag...
20798     The Vexed Question of the Dollar « SGTreport ...
20799    Jerome Hudson Bill Maher Sparks Outrage After ...
Name: content, Length: 20800, dtype: object


In [118]:
# separating the data & label
X = data.drop(columns='label', axis=1)
Y = data['label']

In [119]:
print(X)
print(Y)

          id                                              title  \
0       9182  Alt-Right Gathering Exults in Trump Election W...   
1      11950  Billy Bush, Host on ‘Today,’ Is Suspended by N...   
2       1180                          Let's Eat - The Daily WTF   
3       2810   Stephen King Just Got Real With Trump Supporters   
4      17270  DNC Chair Tom Perez Joins Protesters Defending...   
...      ...                                                ...   
20795  16308  Dem Rep Schiff: Nothing Definitive on Trump-Ru...   
20796   5368  Review: ‘The Night Of’ Takes a Cab Ride to Hel...   
20797  15914  Smoke a Cigar with Dennis Prager to Support Fr...   
20798  17448  The Vexed Question of the Dollar « SGTreport –...   
20799  20000  Bill Maher Sparks Outrage After Telling Ben Sa...   

                  author                                               text  \
0       Joseph Goldstein  WASHINGTON  —   By the time Richard B. Spencer...   
1            John Koblin  Billy Bush,

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

In [120]:
port_stem = PorterStemmer()

In [121]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [122]:
data['content'] = data['content'].apply(stemming)

In [123]:
print(data['content'])

0        joseph goldstein alt right gather exult trump ...
1        john koblin billi bush host today suspend nbc ...
2                            mark bowytz let eat daili wtf
3        natali dickinson stephen king got real trump s...
4        charli spier dnc chair tom perez join protest ...
                               ...                        
20795    pam key dem rep schiff noth definit trump russ...
20796    jame poniewozik review night take cab ride hel...
20797    daniel nussbaum smoke cigar denni prager suppo...
20798    vex question dollar sgtreport corpor propagand...
20799    jerom hudson bill maher spark outrag tell ben ...
Name: content, Length: 20800, dtype: object


In [124]:
#separating the data and label
X = data['content'].values
Y = data['label'].values

In [125]:
print(X)

['joseph goldstein alt right gather exult trump elect nazi era salut new york time'
 'john koblin billi bush host today suspend nbc new york time'
 'mark bowytz let eat daili wtf' ...
 'daniel nussbaum smoke cigar denni prager support free speech'
 'vex question dollar sgtreport corpor propaganda antidot silver gold truth liberti freedom'
 'jerom hudson bill maher spark outrag tell ben sass hous n ga hurt']


In [126]:
print(Y)

[0 0 1 ... 0 1 0]


In [127]:
Y.shape

(20800,)

In [128]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [129]:
print(X)

  (0, 16996)	0.09035028782548123
  (0, 15582)	0.11244627929639132
  (0, 15295)	0.08865104648201927
  (0, 13160)	0.3762211082970048
  (0, 12774)	0.22422056444210306
  (0, 10306)	0.08733439183450624
  (0, 10197)	0.3100529603230123
  (0, 7848)	0.29430715998838014
  (0, 6122)	0.3061345127507276
  (0, 5873)	0.3477240380734747
  (0, 5094)	0.40471817852053477
  (0, 4853)	0.2836646898654677
  (0, 4625)	0.17722625002848225
  (0, 421)	0.3143846031511098
  (1, 16996)	0.10834071015456137
  (1, 15336)	0.3068488064384856
  (1, 15295)	0.10630311826298647
  (1, 14829)	0.3614512866259227
  (1, 10306)	0.10472429319255172
  (1, 10201)	0.3576078713972361
  (1, 8243)	0.373461577916724
  (1, 7824)	0.23096954118372237
  (1, 6992)	0.3437345575586808
  (1, 2111)	0.3670915866731806
  (1, 1520)	0.40936290682425935
  :	:
  (20797, 2708)	0.44832378667201866
  (20798, 16199)	0.3556942635225233
  (20798, 15611)	0.22972628450785343
  (20798, 13856)	0.26644391179014665
  (20798, 13611)	0.3814934009448532
  (20798, 120

Splitting the dataset to training & test data

In [130]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [131]:
model = LogisticRegression()

In [132]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [133]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [134]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9862379807692307


In [94]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [95]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9740384615384615


Making a Predictive System

In [135]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [136]:
print(Y_test[3])

0
